<h1> Optimization electricity consumption </h1>

<h3> Imports </h3>

In [1]:
import math
import random
import copy
import numpy as np
import joblib
import time
import multiprocessing
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

%run preprocess_functions.py
%run Individual.py

<h3> Helper functions </h3>

In [2]:
def calc_consumption_from_planing(plan,mat,day_consumption):
    for house in plan:
        for machine in house.keys():
            for time_slot in house[machine]:
                day_consumption[time_slot] += mat[machine]["power"]
    return day_consumption

<h3> Recuit functions </h3>

In [3]:
#Not Perfect
def initial_solution(empty_plannings,mat):

    for house in range(len(empty_plannings)):
        for machine in empty_plannings[house].keys():
            if type(mat[machine]["nb_time_slot"]) == list:
                nb_time_slots = random.sample(mat[machine]["nb_time_slot"],1)[0]
            else:
                nb_time_slots = mat[machine]["nb_time_slot"]
            
            if machine in ['LL','SL','LV']:
                start_index = random.randint(16,47)
                for i in range(nb_time_slots):
                    empty_plannings[house][machine].append((start_index+i)%48)
            elif machine in ['TV','PL','FO']:
                hours_dict = {"TV":[38,47],"PL":[36,40],"FO":[36,40]}
                start_h,end_h = hours_dict[machine]
                empty_plannings[house][machine] = random.sample(range(start_h, end_h), nb_time_slots)
            else:
                if mat[machine]["is_sequencable"]:
                    for i in range(nb_time_slots):
                        index = random.randint(0,15)
                        while index in empty_plannings[house][machine]:
                            index = random.randint(0,15)
                else:
                    start_index = random.randint(0,15-nb_time_slots)
                    for i in range(nb_time_slots):
                        empty_plannings[house][machine].append(start_index+i)
                

    day_consumption = []
    for i in range(48):
        day_consumption.append(int(0))

    day_consumption = calc_consumption_from_planing(empty_plannings,mat,day_consumption)

    individual = Individual({"start":0,"end":15},empty_plannings,day_consumption)
    

    return individual

In [4]:
def get_neighbour_solution(solution: Individual):
    solution_neigh = copy.deepcopy(solution)
    solution_neigh.mutate()

    return solution_neigh

def cost_function(solution : Individual):
    return solution.getMax()

def metropolis(neighbour_solution, current_solution,temperature):
    return math.exp( -( abs( cost_function(current_solution) - cost_function(neighbour_solution) ) / temperature ) )

def update_temperature(temperature):
    return temperature * 0.99

In [5]:
#TMP
def plot_planning(planning,title):
    m_count = 0
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    for machine in planning.keys():
        m_count += 1
        x = []
        y = []
        for time in planning[machine]:
            x.append(time)
            y.append(machine)
        
        ax.scatter(x,y)
    display(fig)
    clear_output(wait=True)
    plt.pause(0.1)

In [6]:
def recuit_simule(temperature,current_solution):
    best_current_solution = copy.deepcopy(current_solution)
    print_count = 0
    count_iteration_without_improvment = 0

    start_t = time.time()
    while count_iteration_without_improvment < 1000:
        # print_count += 1
        # if print_count%100 == 0:
        #     print("\r",end="")
        #     print("**Recuit",print_count,count_iteration_without_improvment,"| Time :",time.time()-start_t,end="                ")
        #     # plot_planning(best_current_solution.plannings[0],"Testos")
        #     start_t = time.time()

        neighbour_solution = get_neighbour_solution(current_solution)

        if cost_function(neighbour_solution) < cost_function(current_solution) or random.random() < metropolis( neighbour_solution , current_solution , temperature):
            current_solution = neighbour_solution

        if cost_function(current_solution) < cost_function(best_current_solution):
            best_current_solution = copy.deepcopy(current_solution)
            count_iteration_without_improvment = 0
        else:
            count_iteration_without_improvment += 1

        temperature = update_temperature(temperature)

    
    return best_current_solution

<h3> Variables </h3>

In [7]:
matrice_type_puissance_par_demie_heure = {'LV':{"power":float(0.65),"nb_time_slot":2,"is_sequencable":False}, 
                                            'LL':{"power":float(1),"nb_time_slot":2,"is_sequencable":False}, 
                                            'SL':{"power":float(0.125),"nb_time_slot":8,"is_sequencable":False}, 
                                            'TV':{"power":float(0.05),"nb_time_slot":[1,2,3,4,5],"is_sequencable":False}, 
                                            'FG_1':{"power":float(0.1),"nb_time_slot":4,"is_sequencable":False}, 
                                            'CE_1':{"power":float(0.18),"nb_time_slot":12,"is_sequencable":False}, 
                                            'CG':{"power":float(0.1),"nb_time_slot":4,"is_sequencable":False}, 
                                            'FO':{"power":float(0.8),"nb_time_slot":[1,2],"is_sequencable":False}, 
                                            'PL':{"power":float(0.6),"nb_time_slot":[1,2],"is_sequencable":False}, 
                                            'FG_2':{"power":float(0.3),"nb_time_slot":4,"is_sequencable":False}, 
                                            'CE_2':{"power":float(0.25),"nb_time_slot":12,"is_sequencable":False}
                                            }

<h3> Main </h3>

In [8]:
data = load_csv("data/data.csv")
batch_empty_plannings = make_batches("data/data.csv","data/data_network.csv")

In [9]:
# solutions = []
# nb_batches = len(batch_empty_plannings)
# count_batches = 0
# for key in list(batch_empty_plannings.keys())[:3]:
#     batch = batch_empty_plannings[key]

#     print("Init Solution")
#     current_solution = initial_solution(batch,matrice_type_puissance_par_demie_heure)
#     print("Score init :",current_solution.getMax(),"| Batch size :",len(batch)," | Batch number :",str(count_batches)+"/"+str(nb_batches))

#     best_found_solution = recuit_simule(1000, current_solution)
#     curr_best = cost_function(best_found_solution)

#     # print(best_found_solution.day_consumption)
#     print("Score optimized",cost_function(best_found_solution))
#     solutions.append(best_found_solution)
#     count_batches += 1




In [10]:
# joblib.dump(best_found_solution.plannings,"out/Plannings.joblib")
# joblib.dump(best_found_solution.day_consumption,"out/Day_Consumption.joblib")
# joblib.dump(best_found_solution,"out/Solution.joblib")
# joblib.dump(solutions,"out/Solutions.joblib")

In [11]:
def worker(procnum,batch,key,return_dict):
    print("Init Solution, process :", procnum)
    init_solution = initial_solution(batch,matrice_type_puissance_par_demie_heure)
    # print("Score init",cost_function(init_solution),", process :", procnum)
    score_init = cost_function(init_solution)

    best_found_solution = recuit_simule(1000, init_solution)
    curr_best = cost_function(best_found_solution)

    # print(best_found_solution.day_consumption)
    print("Score init/optimized",score_init,cost_function(best_found_solution),", process :", procnum)
    return_dict[key] = [best_found_solution,init_solution]
    print("Process Done :",procnum)


manager = multiprocessing.Manager()
return_dict = manager.dict()
jobs = []

nb_batches = len(batch_empty_plannings)
count_batches = 0
for key in list(batch_empty_plannings.keys())[:10]:
    batch = batch_empty_plannings[key]
    p = multiprocessing.Process(target=worker, args=(count_batches,batch,key,return_dict))
    jobs.append(p)
    p.start()
    count_batches += 1
    

for proc in jobs:
    proc.join()

solutions = return_dict.values()
print(len(solutions))

Init Solution, process : Init Solution, process :0 1

Init Solution, process : Init Solution, process :2Init Solution, process :Init Solution, process :Init Solution, process : 
  534 Init Solution, process :


 6
7Init Solution, process :
Init Solution, process :  98

Score init/optimized 127.74999999999957 120.475 , process : 6
Process Done : 6
Score init/optimized 122.94999999999959 114.55 , process : 4
Process Done : 4
Score init/optimized 133.74999999999972 118.45 , process : 3
Process Done : 3
Score init/optimized 126.74999999999963 112.575 , process : 9
Process Done : 9
Score init/optimized 127.82499999999963 108.1 , process : 2
Process Done : 2
Score init/optimized 138.72499999999977 110.725 , process : 5
Process Done : 5
Score init/optimized 127.62499999999959 106.925 , process : 7
Process Done : 7
Score init/optimized 132.67499999999964 108.7 , process : 1
Process Done : 1
Score init/optimized 139.2249999999997 112.5 , process : 0
Process Done : 0
Score init/optimized 134.599

In [12]:
# joblib.dump(solutions,"out/Solution_f10.joblib")

['out/Solution_f10.joblib']

In [ ]:
matrice_machine = {'LV':{"power":float(0.65),"nb_time_slot":2,"is_sequencable":False}, 
                    'LL':{"power":float(1),"nb_time_slot":2,"is_sequencable":False}, 
                    'SL':{"power":float(0.125),"nb_time_slot":8,"is_sequencable":False}, 
                    'FG_1':{"power":float(0.1),"nb_time_slot":4,"is_sequencable":False}, 
                    'CE_1':{"power":float(0.18),"nb_time_slot":12,"is_sequencable":True}, 
                    'CG':{"power":float(0.1),"nb_time_slot":4,"is_sequencable":False}, 
                    'FG_2':{"power":float(0.3),"nb_time_slot":4,"is_sequencable":False}, 
                    'CE_2':{"power":float(0.25),"nb_time_slot":12,"is_sequencable":True}
                                            }

def get_start_dir(full_indic,nb_time_slot,limit_up,limit_down,empty_day):
    limit_up_true = limit_up+1
    indic_index = -1
    empty_index = -1
    for i in range(len(full_indic)):
        if limit_up_true-sum(full_indic[i][:limit_up_true]) >= nb_time_slot:
            indic_index = i
            break
        elif limit_up_true-sum(full_indic[i][:limit_up_true]) != 0 :
            empty_index = full_indic[i][:limit_up_true].index(0)
    if indic_index == -1:
        full_indic.append(empty_day.copy())

    if empty_index != -1:
        start = empty_index
        if empty_index == limit_down:
            direction = 1
        elif empty_index == limit_up:
            direction = -1
        else:
            if empty_index+nb_time_slot > limit_up:
                direction = -1
            elif empty_index-nb_time_slot < limit_down:
                direction = 1
            else:
                direction = random.sample([-1,1],1)[0]
    else:
        direction = random.sample([-1,1],1)[0]
        if direction == 1:
            start = limit_down
        else:
            start = limit_up

    return [indic_index,direction,start,full_indic]

def stupid_solution(empty_plannings,mat):
    HC = {"start":0,"end":16}
    empty_day = []
    for i in range(48):
        empty_day.append(0)

    full_indic = [empty_day.copy()]
    for house in range(len(empty_plannings)):
        print("HOUSE",house)
        machine_list = []
        for machine in empty_plannings[house].keys():
            if not machine in ['TV','PL','FO']:
                machine_list.append([machine,mat[machine]["nb_time_slot"]])

        machine_list.sort(key=lambda x: x[1],reverse=True)

        for machine in machine_list:
            nb_time_slot = mat[machine[0]]["nb_time_slot"]
            if machine[0] in ['LL','LV',"SL"]:
                [indic_index,direction,start,full_indic] = get_start_dir(full_indic,nb_time_slot,47,0,empty_day)
            else:
                [indic_index,direction,start,full_indic] = get_start_dir(full_indic,nb_time_slot,15,0,empty_day)
            
            for i in range(nb_time_slot):
                shift = 0
                print("MA",machine,start,direction)
                print("FI",len(full_indic),indic_index,full_indic[indic_index])
                print("SHIFT",len(full_indic[indic_index]),start+(i*direction)+(shift*direction))
                while full_indic[indic_index][start+(i*direction)+(shift*direction)] == 1:
                    shift += 1
                empty_plannings[house][machine[0]].append(start+(i*direction)+(shift*direction))
                full_indic[indic_index][start+(i*direction)+(shift*direction)] = 1
                if sum(full_indic[indic_index]) == 48:
                    full_indic.remove(full_indic[indic_index])
        # print("EP",empty_plannings[house])
        # for f in full_indic:
        #     print("FI",f)
                
    day_consumption = []
    for i in range(48):
        day_consumption.append(int(0))

    day_consumption = calc_consumption_from_planing(empty_plannings,mat,day_consumption)

    individual = Individual({"start":0,"end":15},empty_plannings,day_consumption)
    

    return individual

data = load_csv("data/data.csv")
batch_empty_plannings = make_batches("data/data.csv","data/data_network.csv")

sol = stupid_solution(batch_empty_plannings["PL1111"],matrice_machine)
print(sol.day_consumption)

In [159]:
def get_start_and_dir(power_indic,nb_time_slot,limit_up,limit_down):
    power_indic_sorted = power_indic.copy()
    power_indic_sorted.sort()
    direction = 1

    for curr_min in power_indic_sorted:
        index = power_indic.index(curr_min)
        if index >= limit_down and index <= limit_up:
            if index+nb_time_slot-1 <= limit_up:
                direction = 1
            else:
                direction = -1
            break

    return [index,direction]

def not_so_stupid_solution(empty_plannings,mat,fixed_machines):
    HC = {"start":0,"end":16}

    power_indic = []
    for i in range(48):
        power_indic.append(0)
    calc_consumption_from_planing(empty_plannings,mat,power_indic)

    for house in range(len(empty_plannings)):
        print("HOUSE",house)
        machine_list = []
        for machine in empty_plannings[house].keys():
            if not machine in fixed_machines:
                machine_list.append([machine,mat[machine]["nb_time_slot"]])

        machine_list.sort(key=lambda x: x[1],reverse=True)

        for machine in machine_list:
            nb_time_slot = mat[machine[0]]["nb_time_slot"]
            if machine[0] in ['LL','LV','SL','TV','PL','FO']:
                [start,direction] = get_start_and_dir(power_indic,nb_time_slot,47,16)
            else:
                [start,direction] = get_start_and_dir(power_indic,nb_time_slot,15,0)

            for i in range(nb_time_slot):
                power_indic[start+(i*direction)] += mat[machine[0]]["power"]
                empty_plannings[house][machine[0]].append(start+(i*direction))
            empty_plannings[house][machine[0]].sort()
            
            
        print("EP",empty_plannings[house])
        print("PI",power_indic)
                
    day_consumption = []
    for i in range(48):
        day_consumption.append(int(0))

    day_consumption = calc_consumption_from_planing(empty_plannings,mat,day_consumption)
    
    return [empty_plannings,day_consumption]

def init_plannings(batch_empty_plannings,fixed_machines):
    for batch_key in batch_empty_plannings:
        for i in range(len(batch_empty_plannings[batch_key])):
            planning = batch_empty_plannings[batch_key][i]
            for machine in planning:
                if machine in fixed_machines.keys():
                    [start,end] = fixed_machines[machine]
                    for j in range(end-start):
                        batch_empty_plannings[batch_key][i][machine].append(start+j)
    
    return batch_empty_plannings

PR = 0.5
matrice_machine = {'LV':{"power":float(0.65),"nb_time_slot":2,"is_sequencable":False}, 
                'LL':{"power":float(1),"nb_time_slot":2,"is_sequencable":False}, 
                'SL':{"power":float(0.125),"nb_time_slot":8,"is_sequencable":False}, 
                'TV':{"power":float(0.05*PR),"nb_time_slot":5,"is_sequencable":False}, 
                'FG_1':{"power":float(0.1),"nb_time_slot":4,"is_sequencable":False}, 
                'CE_1':{"power":float(0.18),"nb_time_slot":12,"is_sequencable":False}, 
                'CG':{"power":float(0.1),"nb_time_slot":4,"is_sequencable":False}, 
                'FO':{"power":float(0.8*PR),"nb_time_slot":2,"is_sequencable":False}, 
                'PL':{"power":float(0.6*PR),"nb_time_slot":2,"is_sequencable":False}, 
                'FG_2':{"power":float(0.3),"nb_time_slot":4,"is_sequencable":False}, 
                'CE_2':{"power":float(0.25),"nb_time_slot":12,"is_sequencable":False}
                }

# fixed_machines = {'TV':[36,47],'PL':[38,42],'FO':[38,42]}
fixed_machines = {}

data = load_csv("data/data.csv")
batch_empty_plannings = make_batches("data/data.csv","data/data_network.csv")

batch_init_plannings = init_plannings(batch_empty_plannings,fixed_machines)

[empty_plannings,day_consumption] = not_so_stupid_solution(batch_empty_plannings["PL1111"],matrice_machine,fixed_machines)
print(day_consumption)
print(max(day_consumption))

HOUSE 0
EP {'LV': [11, 12], 'LL': [11, 12], 'TV': [1, 2, 3, 4, 5], 'FG_1': [10, 11, 12, 13], 'CE_1': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 'FO': [11, 12], 'PL': [11, 12], 'FG_2': [11, 12, 13, 14], 'CE_2': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]}
PI [0.18, 0.455, 0.455, 0.455, 0.455, 0.455, 0.43, 0.43, 0.43, 0.43, 0.53, 3.1799999999999997, 2.9999999999999996, 0.4, 0.3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
HOUSE 1
EP {'LV': [29, 30], 'LL': [31, 32], 'SL': [16, 17, 18, 19, 20, 21, 22, 23], 'TV': [24, 25, 26, 27, 28], 'FG_1': [0, 1, 2, 3], 'CE_1': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 'FO': [33, 34], 'PL': [35, 36], 'FG_2': [12, 13, 14, 15]}
PI [0.28, 0.555, 0.555, 0.555, 0.635, 0.635, 0.61, 0.61, 0.61, 0.61, 0.71, 3.36, 3.4799999999999995, 0.8800000000000001, 0.78, 0.48, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.025, 0.025, 0.025, 0.025, 0.025, 0.65, 0.65, 1.0, 1.0, 0.4, 0.4, 0.3, 0.3, 0, 0, 0, 0, 0, 0, 